In [36]:
# Obtaining the significant regions based on literature

import pandas as pd

temp_list = []

with open("train_call.txt", 'r') as temp:
    for line in temp:
        temp_list.append(line.split())

columns_temp = temp_list[0]
columns_temp = [x.replace("\"", "") for x in columns_temp]

df_train = pd.DataFrame(temp_list[1:], columns=columns_temp)

sig_reg_list = []

with open("significant_regions.csv", 'r') as sig_regs:
    next(sig_regs)
    for line in sig_regs:
        line_temp = line.split(",")
        sig_reg_list.append([int(line_temp[2].split()[0]), int(line_temp[2].split()[2])])
        
start_regs = df_train["Start"].values.tolist()
end_regs = df_train["End"].values.tolist()

start_regs = [int(i) for i in start_regs]
end_regs = [int(i) for i in end_regs]

sig_labels = []

for region in sig_reg_list:
    for i in range(len(start_regs) - 1):
        if region[0] > start_regs[i] and region[1] < end_regs[i]:
            print("Start region = {0} - {1}".format(start_regs[i], end_regs[i]))
            print("Significant region = {0} - {1}\n".format(region[0], region[1]))
            sig_labels.append(i)
            if region[1] > start_regs[i + 1]:
                sig_labels.append(i + 1)

Start region = 42754729 - 43839204
Significant region = 43044295 - 43125364

Start region = 41616449 - 43362384
Significant region = 43044295 - 43125364

Start region = 41249898 - 46752789
Significant region = 43044295 - 43125364

Start region = 42710489 - 45504577
Significant region = 43044295 - 43125364

Start region = 41620761 - 43330593
Significant region = 43044295 - 43125364

Start region = 42836838 - 44732805
Significant region = 43044295 - 43125364

Start region = 43002620 - 45737633
Significant region = 43044295 - 43125364

Start region = 42917603 - 43133142
Significant region = 43044295 - 43125364

Start region = 41982016 - 43404918
Significant region = 43044295 - 43125364

Start region = 42744201 - 43936926
Significant region = 43044295 - 43125364

Start region = 42053349 - 45262389
Significant region = 43044295 - 43125364

Start region = 42076777 - 43555063
Significant region = 43044295 - 43125364

Start region = 41676219 - 44156737
Significant region = 43044295 - 43125364


In [35]:
len(sig_labels)

453

In [18]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt

#Loading data
df = pd.read_csv('Processed_data_2.csv')

# X for features Y for breast cancer subtype where 1 = HER2+, 2 = HR+, 3 = Triple Neg
X = df.drop(['Unnamed: 0', 'Sample','Subgroup'], axis=1)
Y = df['Subgroup']

patients = X.values.tolist()
labels = Y.values.tolist()

## FEATURE SELECTION

# we need to shift the features by 1 since the chi2 function does not take non-neg values
patients_shift = []

for patient in patients:
    x = []
    for feature in patient:
        x.append(feature + 1)
    patients_shift.append(x)

chi, p_val = chi2(patients_shift, Y)

X_feature_selected = df

p_value = 0.05
for i, val in enumerate(p_val):
#     if val > 0.01:
    if val >= p_value and i not in sig_labels:
        X_feature_selected = X_feature_selected.drop(["V{}".format(i + 1)], axis=1)
        
X = X_feature_selected.drop(['Unnamed: 0', 'Sample','Subgroup'], axis=1)
patients = X.values.tolist()

In [19]:
print("Number of features = {}".format(len(patients[0])))

Number of features = 138


In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

def classification_report_with_accuracy_score(y_true, y_pred):
    originalclass.extend(y_true)
    predictedclass.extend(y_pred)
    return accuracy_score(y_true, y_pred) 

def evaluate_model(model, features, labels):
    scores = cross_val_score(model, features, labels, cv=5, scoring=make_scorer(classification_report_with_accuracy_score))
    return scores

In [22]:
# Random Forest

originalclass = []
predictedclass = []

random_forest = RandomForestClassifier(n_estimators=100)
temp = evaluate_model(random_forest, X, Y)
print(classification_report(originalclass, predictedclass)) 

              precision    recall  f1-score   support

           1       0.88      0.91      0.89        32
           2       0.79      0.75      0.77        36
           3       0.76      0.78      0.77        32

    accuracy                           0.81       100
   macro avg       0.81      0.81      0.81       100
weighted avg       0.81      0.81      0.81       100



In [23]:
# Naive Bayes

originalclass = []
predictedclass = []

gnb = GaussianNB()
temp = evaluate_model(gnb, X, Y)
print(classification_report(originalclass, predictedclass))

              precision    recall  f1-score   support

           1       1.00      0.84      0.92        32
           2       0.78      0.81      0.79        36
           3       0.67      0.75      0.71        32

    accuracy                           0.80       100
   macro avg       0.82      0.80      0.81       100
weighted avg       0.82      0.80      0.80       100



In [24]:
originalclass = []
predictedclass = []

mlp = MLPClassifier(solver='lbfgs', activation="relu", alpha=1e-5,
                    hidden_layer_sizes=(len(patients[0]),), max_iter=1000)

temp = evaluate_model(mlp, X, Y)
print(classification_report(originalclass, predictedclass))

              precision    recall  f1-score   support

           1       0.97      0.97      0.97        32
           2       0.84      0.89      0.86        36
           3       0.83      0.78      0.81        32

    accuracy                           0.88       100
   macro avg       0.88      0.88      0.88       100
weighted avg       0.88      0.88      0.88       100



In [25]:
originalclass = []
predictedclass = []

mlp = MLPClassifier(solver='lbfgs', activation="relu", alpha=1e-5,
                    hidden_layer_sizes=(len(patients[0]),len(patients[0])), max_iter=1000)

temp = evaluate_model(mlp, X, Y)
print(classification_report(originalclass, predictedclass))

              precision    recall  f1-score   support

           1       0.94      0.94      0.94        32
           2       0.82      0.89      0.85        36
           3       0.83      0.75      0.79        32

    accuracy                           0.86       100
   macro avg       0.86      0.86      0.86       100
weighted avg       0.86      0.86      0.86       100



In [26]:
originalclass = []
predictedclass = []

mlp = MLPClassifier(solver='lbfgs', activation="relu", alpha=1e-5,
                    hidden_layer_sizes=(len(patients[0]),len(patients[0]),len(patients[0])), max_iter=1000)

temp = evaluate_model(mlp, X, Y)
print(classification_report(originalclass, predictedclass))

              precision    recall  f1-score   support

           1       0.97      0.91      0.94        32
           2       0.82      0.86      0.84        36
           3       0.81      0.81      0.81        32

    accuracy                           0.86       100
   macro avg       0.86      0.86      0.86       100
weighted avg       0.86      0.86      0.86       100

